# Test the quality of the data of Camera Motion

In [55]:
import cv2
from pathlib import Path
import pandas as pd

Import video

In [56]:
video_number = "3"
# Define the relative path to the video file
notebook_dir = Path().resolve()
project_root = notebook_dir.parent.parent
video_path = (
    project_root
    / "data"
    / f"recording_{video_number}"
    / f"Recording_{video_number}.mp4"
)
video_path = str(video_path)

# Load the video
cap = cv2.VideoCapture(video_path)

# Check
print(
    f"Opened: {cap.isOpened()}, FPS: {cap.get(cv2.CAP_PROP_FPS)}, Total Frames: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}"
)

Opened: True, FPS: 59.94005994005994, Total Frames: 227.0


Import data

In [57]:
# Define the iinput path for the lines CSV file
motion = (
    project_root
    / "notebook"
    / "lane_detection"
    / "intermediate_data"
    / "background_motion"
    / f"motion_{video_number}.csv"
)
df = pd.read_csv(motion)

Process the motion

In [58]:
# Define a function to remove outliers and replace them with 0
def remove_outliers(series, threshold=3):
    # Calculate the first quartile (Q1) and third quartile (Q3)
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)

    # Compute the interquartile range (IQR)
    IQR = Q3 - Q1

    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR

    # Replace values outside the bounds with 0, otherwise keep the value
    return series.apply(lambda x: 0 if x < lower_bound or x > upper_bound else x)


# Apply the function to 'dx' and 'dy'
df["dx"] = remove_outliers(df["dx"])
df["dy"] = remove_outliers(df["dy"])

print(df)

               dx            dy
0    2.039125e-12  1.108998e-12
1    1.359889e-12  4.476052e-13
2    0.000000e+00  0.000000e+00
3   -1.266178e+01  0.000000e+00
4    1.465242e+01  0.000000e+00
..            ...           ...
221  3.414250e+00  1.546667e+00
222  1.636811e+00  5.689573e-01
223  3.211425e+00  5.978990e-01
224  3.204671e+00  4.091686e-01
225  5.896944e+00  1.380317e+00

[226 rows x 2 columns]


Select a point in the first frame

In [59]:
# Get the width and height of the video frame
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Calculate the central point
# point = (frame_width // 2, frame_height // 2)

point = (1400, 540)

print(f"Central Point: {point}")

Central Point: (1400, 540)


Add to the point the detected motion

In [60]:
# Initialize the starting point
current_point = list(point)

# Create a list to store the updated points for each frame
updated_points = []

# Iterate through the dataframe to add the motion to the point
for _, row in df.iterrows():
    motion_x, motion_y = row["dx"], row["dy"]
    current_point[0] += motion_x
    current_point[1] += motion_y
    updated_points.append(tuple(current_point))

# # Print the updated points
# print(updated_points)

Draw the point on the video on each frame

In [61]:
# Reset the video to the beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Define the codec and create a VideoWriter object to save the modified frames
output_path = project_root / "data" / f"recording_{video_number}" / "Motion_test.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use 'mp4v' codec for MP4 format
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(str(output_path), fourcc, fps, (frame_width, frame_height))


# Loop through each frame in the video
frame_index = 0
while frame_index < len(updated_points):
    ret, video_frame = cap.read()
    if not ret:
        print("End of video or failed to read the frame at iteration", frame_index)
        break
    # print(f"Processing frame {frame_index}")

    modified_frame = video_frame.copy()

    # Draw the updated point on the frame
    updated_point = updated_points[frame_index]
    cv2.circle(
        modified_frame,
        (int(updated_point[0]), int(updated_point[1])),
        radius=5,
        color=(0, 0, 255),
        thickness=-1,
    )

    # Write the modified frame to the output video
    out.write(modified_frame)

    # Increment the frame index
    frame_index += 1

# Release the video capture and writer objects
# cap.release()
out.release()

print(f"Adjusted video saved to {output_path}")

Adjusted video saved to C:\Users\miche\OneDrive\Documenti\GitHub\bowling-analysis\data\recording_3\Motion_test.mp4
